# Sequential Workflow: Content Generation Pipeline

A workflow with three stages: outline, draft, and edit. Each stage produces typed output that feeds into the next. The orchestrator controls execution flow while specialist prompts handle each phase.

In [ ]:
from pydantic import BaseModel, Field
from agentic_patterns.core.agents import get_agent, run_agent

## Stage Output Types

Each stage produces structured output. This enforces contracts between stages and makes the workflow's data flow explicit.

In [ ]:
class Outline(BaseModel):
    title: str = Field(description="Article title")
    sections: list[str] = Field(description="Section headings in order")
    key_points: list[str] = Field(description="Main points to cover")

class Draft(BaseModel):
    content: str = Field(description="Full article text")
    word_count: int = Field(description="Approximate word count")

class EditedArticle(BaseModel):
    content: str = Field(description="Final edited article")
    changes_made: list[str] = Field(description="Summary of edits applied")

## Shared State

A simple container accumulates outputs from each stage. The orchestrator passes relevant state to each specialist.

In [ ]:
class WorkflowState(BaseModel):
    topic: str
    outline: Outline | None = None
    draft: Draft | None = None
    final: EditedArticle | None = None

state = WorkflowState(topic="The benefits of morning exercise routines")

## Stage 1: Outline

The outline agent receives the topic and produces a structured plan. The `output_type` parameter ensures the response conforms to the Outline schema.

In [ ]:
outline_agent = get_agent(
    output_type=Outline,
    system_prompt="You are an outline specialist. Create clear, logical article structures."
)

outline_prompt = f"Create an outline for a short article about: {state.topic}"

agent_run, _ = await run_agent(outline_agent, outline_prompt)
state.outline = agent_run.result.output

print(f"Title: {state.outline.title}")
print(f"Sections: {state.outline.sections}")
print(f"Key points: {state.outline.key_points}")

## Stage 2: Draft

The draft agent receives the outline and writes the full content. State from the previous stage flows into the prompt.

In [ ]:
draft_agent = get_agent(
    output_type=Draft,
    system_prompt="You are a content writer. Write engaging, informative articles based on outlines."
)

draft_prompt = f"""Write a short article (~300 words) based on this outline:

Title: {state.outline.title}
Sections: {', '.join(state.outline.sections)}
Key points to cover: {', '.join(state.outline.key_points)}"""

agent_run, _ = await run_agent(draft_agent, draft_prompt)
state.draft = agent_run.result.output

print(f"Word count: {state.draft.word_count}")
print(f"\nDraft:\n{state.draft.content[:500]}...")

## Stage 3: Edit

The editor agent refines the draft. It receives both the outline (for reference) and the draft content.

In [ ]:
editor_agent = get_agent(
    output_type=EditedArticle,
    system_prompt="You are an editor. Improve clarity, fix errors, and enhance readability while preserving the author's voice."
)

edit_prompt = f"""Edit and improve this article. Make it clearer and more engaging.

Original outline for reference:
- Title: {state.outline.title}
- Key points: {', '.join(state.outline.key_points)}

Draft to edit:
{state.draft.content}"""

agent_run, _ = await run_agent(editor_agent, edit_prompt)
state.final = agent_run.result.output

print("Changes made:")
for change in state.final.changes_made:
    print(f"  - {change}")
print(f"\nFinal article:\n{state.final.content}")

## The Complete Pipeline as a Function

Encapsulating the workflow in a function makes it reusable. The orchestrator controls the sequence while each stage remains a focused specialist call.

In [ ]:
async def content_pipeline(topic: str) -> WorkflowState:
    """Run the complete content generation workflow."""
    state = WorkflowState(topic=topic)
    
    # Stage 1: Outline
    outline_agent = get_agent(output_type=Outline, system_prompt="Create clear article outlines.")
    agent_run, _ = await run_agent(outline_agent, f"Create an outline for: {topic}")
    state.outline = agent_run.result.output
    
    # Stage 2: Draft
    draft_agent = get_agent(output_type=Draft, system_prompt="Write engaging articles from outlines.")
    agent_run, _ = await run_agent(draft_agent, f"Write ~300 words based on: {state.outline.model_dump_json()}")
    state.draft = agent_run.result.output
    
    # Stage 3: Edit
    editor_agent = get_agent(output_type=EditedArticle, system_prompt="Edit for clarity and engagement.")
    agent_run, _ = await run_agent(editor_agent, f"Edit this article: {state.draft.content}")
    state.final = agent_run.result.output
    
    return state


result = await content_pipeline("Why reading fiction improves empathy")
print(f"Generated article: {result.final.content[:200]}...")